# Dynamic optimization

In [ ]:
from platform import python_version
print(python_version())

## Index
1. starting from single shots
    https://media.xiph.org/video/derf/
    ftp://svtopencontent.svt.se
2. coding raw videos in AVC, HEVC, VP9 (in seguito ed eventualmente AV1 e VVC) using:
    - ffmpeg library for AVC
    - TODO
    
3. script per generazione automatica di codifiche multiple, in python, includendo anche risoluzioni fino a 4k (sulle 10 risoluzioni proposte dal paper, selezionarne 5 e aggiungerne 2 oltre l'HD) con convex hull RD dei singoli shot e visualizzazione in MATLAB

4. valutazione risultati con PSNR, MSSIM e soprattutto VMAF e comparazione per contenuto dei singoli shot

5. memorizzazione risultati su csv

6. divisione video in spezzoni con ffmpeg

7. script python per generazione dell'intero video codificato seguendo i passi descritti finora per ogni shot e con convex hull generale; test su 3 titoli: ritmo lento, azione e animazione

8. valutazione risultati come punto e. anche in base alla durata degli shot

In [18]:
#show info
!ffprobe -hide_banner test_vids/smithy.mov

test_vids/smithy.mov: No such file or directory


In [19]:
!ffprobe -hide_banner test_vids/encoded/smithyAVC_1280_cf23.mp4

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'test_vids/encoded/smithyAVC_1280_cf23.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:30.00, start: 0.000000, bitrate: 3682 kb/s
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 1280x720, 3677 kb/s, 50 fps, 50 tbr, 12800 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]


[1920  1080; 1536  864; 1216  684; 960  540; 768  432; 608  342; 480  270; 384  216; 320  180; 256  144]

In [ ]:
import numpy as np
import subprocess
source = "smithy"
res_widths = np.array([3840, 2560, 1920, 1280, 1024, 640])
res_heights = np.array([2160, 1440, 1080, 720, 576, 360])
for i in range(1, res_widths.size):
    path = "test_vids/downsampled/" + source + "_" + str(res_widths[i-1]) + ".yuv"
    out = "test_vids/downsampled/" + source + "_" + str(res_widths[i]) + ".yuv"
    command = f"ffmpeg -f rawvideo -video_size {res_widths[i-1]}x{res_heights[i-1]} -r 50 -pixel_format yuv420p \
    -i {path} -f rawvideo -c:v rawvideo -filter:v scale={res_widths[i]}x{res_heights[i]} -pix_fmt yuv420p {out}"
    subprocess.call(command, shell=True)

In [ ]:
import os
import subprocess
folder = 'test_vids/downsampled'
source = "smithy"
res_widths = [3840, 2560, 1920, 1280, 1024, 640]
res_heights = [2160, 1440, 1080, 720, 576, 360]
quality = [1, 11, 23, 34, 50]
for file in os.listdir(folder):
    path = os.path.join(folder, file)
    if str(file).endswith('.yuv'):
        current_res = str(file)[:-4].strip(source + "_")
        current_index = res_widths.index(int(current_res))
        for current_crf in range(0,5):
            out = "test_vids/encoded/" + source + "AVC_" + str(current_res) + "_cf" + str(quality[current_crf]) + ".mp4"
            to_avc = f"ffmpeg -f rawvideo -video_size {res_widths[current_index]}x{res_heights[current_index]} \
            -r 50 -pixel_format yuv420p -i {path} -c:v libx264 -crf {quality[current_crf]} {out}"
            subprocess.call(to_avc, shell=True)


In [ ]:
!ffplay -f rawvideo -video_size 640x360 test_vids/downsampled/smithy_640.yuv

In [43]:
import subprocess
import json
command="ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json test_vids/encoded/smithyAVC_3840_cf1.mp4"
cout = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout
dict = json.loads(cout)  # Convert data from JSON string to dictionary
file_name = str(dict['format']['filename'])
print(file_name)
cbr = file_name.split("/")[-1]
print(cbr)

test_vids/encoded/smithyAVC_3840_cf1.mp4
smithyAVC_3840_cf1.mp4


In [36]:
import subprocess
import json
command="ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json test_vids/encoded/smithyAVC_3840_cf34.mp4"
cout = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout
dict = json.loads(cout)  # Convert data from JSON string to dictionary
bitrate = int(dict['streams'][0]['bit_rate'])  # Get the bitrate.
print(dict)

{'programs': [], 'streams': [{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'High', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 3840, 'height': 2160, 'coded_width': 3840, 'coded_height': 2160, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 2, 'pix_fmt': 'yuv420p', 'level': 52, 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '50/1', 'avg_frame_rate': '50/1', 'time_base': '1/12800', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 384000, 'duration': '30.000000', 'bit_rate': '5070082', 'bits_per_raw_sample': '8', 'nb_frames': '1500', 'extradata_size': 42, 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impaired': 0, 'clean_effects': 0, 'attached_pic': 0, 'timed_thumbnails': 0, 'captions':